In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Ejercicio 1
Parte 1 (imágenes en /white_patchy /coord_cromaticas)
1) Implementar el algoritmo de pasaje a coordenadas cromáticas para librarnos de las variaciones de contraste

In [2]:
# Cargar las imágenes cromáticas
imagen_original_1 = cv2.imread('coord_cromaticas/CoordCrom_1.png')
imagen_original_1 = cv2.cvtColor(imagen_original_1, cv2.COLOR_BGR2RGB)
imagen_original_2 = cv2.imread('coord_cromaticas/CoordCrom_2.png')
imagen_original_2 = cv2.cvtColor(imagen_original_2, cv2.COLOR_BGR2RGB)
imagen_original_3 = cv2.imread('coord_cromaticas/CoordCrom_3.png')
imagen_original_3 = cv2.cvtColor(imagen_original_3, cv2.COLOR_BGR2RGB)

